#MAIN


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd drive
%ls
%cd My\ Drive
# %ls
%cd 'Hand Tracking'
%ls
# print('=======================================')
# %cd ASL_dataset/
# %ls

In [ ]:
!pip install dtaidistance

     |████████████████████████████████| 327kB 4.7MB/s 
  Created wheel for dtaidistance: filename=dtaidistance-1.2.3-cp36-cp36m-linux_x86_64.whl size=624188 sha256=2ddb2b2cc6ad7095c32b44f9ac978b33761ab1004e9a6337353a0f3fe4e95755
  Stored in directory: /root/.cache/pip/wheels/9f/9a/74/42c5b9dc714bb5a70b426df08cc860e6a741bc199004873e1b
Successfully built dtaidistance


In [ ]:
import cv2
import hand_tracker
from dtaidistance import dtw,dtw_ndim
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pickle
import sys
import os
# from videocapture import take_vid
# sys.path.append('/home/rakshak/anaconda3/envs/tensorflow/lib/python3.7/site-packages')

VERBOSE = False
SHOW_FRAMES = False
DISPLAY_LANDMARKS = True
NORMALIZATION = False
STILL_IMAGE = False
ONE_SHOT_LEARNING = False


class MainPart:

    def hand_point_gen(self, img):

        # box_shift determines
        palm_model_path = "models/palm_detection.tflite"
        landmark_model_path = "models/hand_landmark.tflite"
        anchors_path = "data/anchors.csv"
        detector = hand_tracker.HandTracker(palm_model_path, landmark_model_path, anchors_path,
                           box_shift=0.2, box_enlarge=1, verbose=VERBOSE)
        kp, box = detector(img)
        if box is not None:
            for i in range(len(box)):
                for j in range(len(box[0])):
                    box[i][j] = int(box[i][j])
        return kp,box



    def video_process(self, name):
        points_list=[]
        cap = cv2.VideoCapture(name)
        amount_of_frames = cap.get(7)
        #print(amount_of_frames)

        start_frame_number = int(amount_of_frames*4.5/10)
        end_frame_number = int(amount_of_frames*4.3/10)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
        #print(name)
        first = True
        x_shift ,y_shift = 0,0
        cnt = 0

        while(True) and cnt < amount_of_frames - start_frame_number - end_frame_number:
            cnt += 1
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                break

            if '.mov' in name:
                cut_frame = frame.shape[0]//2
                frame = frame[:(cut_frame-70), :]
            #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            kp,box = self.hand_point_gen(frame[:,:,::-1]) # <=================== change here 
            normalized_kp = 0
            #print(box)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if kp is not None:
                if VERBOSE:
                    print(kp.shape)

                normalized_kp = kp
                if NORMALIZATION:
                    if first:
                        height, width = frame.shape[:2]
                        x_shift = width - ((box[2][0]+box[0][0])//2) # add the shift
                        y_shift = height - ((box[2][1]+box[0][1])//2)
                        first = False
                        temp = kp[:,:]+[y_shift, x_shift]
                    normalized_kp = kp[:,:]+[y_shift, x_shift]
                points_list.append(normalized_kp)

                if DISPLAY_LANDMARKS:
                    try:
                        cv2.rectangle(frame,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,255,0),3)
                        for i in range(len(kp[:,0])):
                            if VERBOSE:
                                print(kp[i,1],kp[i,0])
                            frame[int(kp[i,1]),int(kp[i,0])] = (0,0,255)
                            cv2.circle(frame,(int(kp[i,0]),int(kp[i,1])),5,(255,0,0),1)
                            cv2.circle(frame,(int(box[0][0]),int(box[0][1])),5,(0,0,255),1)
                            cv2.circle(frame,(int(((box[2][0]+box[0][0])//2)),int(((box[2][1]+box[0][1])//2))),5,(0,0,255),1)
                            #cv2.circle(frame,(int(box[2][1]),int(box[0][1])),5,(255,0,0),1)
                    except Exception as e:
                        print(e)

                
            if SHOW_FRAMES:
                cv2.imshow('frame', frame)

            if STILL_IMAGE:
                break #For still images
        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()
        #print(cnt)
        return points_list


    def load_data(self, list_of_vid_names):
        x = list()
        for i in list_of_vid_names:
            x.append(self.video_process(i))
        
        for i in range(len(x)):
            x[i] = np.array(x[i])
        
        return x


    def change_format(self, x):
        vid = np.zeros((21,x.shape[0],2)) # frames,21,2 => 21,frames,2
        cnt = 0
        for i in range(21):
            for j in range(x.shape[0]):
                if j == 0:
                    shifting = x[j,i]
                vid[i,j] = x[j,i] - shifting
        return vid

    def dis_sep(self, f, s, retain_list=False):
        first_vid = self.change_format(f)
        second_vid = self.change_format(s)
        dis_list = []
        for i in range(21):
            dis_list.append(dtw_ndim.distance(first_vid[i], second_vid[i]))

        if retain_list:
            return dis_list

        return sum(dis_list)/21

    def load_vids_from_paths(self,path, option, custom=False):
        print('------------------'+option+'------------------')
        path = path+option
        paths = self.get_list_of_vids_from_folder(path)
        data = self.load_data(paths)

        if custom:
            classes = [x.rsplit('/', 1)[1].replace('.avi', '') for x in paths]
            return data, len(data), classes

        return data, len(data)

    def customMain(self, a_class_option=None):
        
        size_of_1 = 0
        data1 = []
        a_class = ['OTHER', 'BACHELOR', 'AIRPLANE', 'ADULT-TALL'] #,'AND'] #'#ALL']#, 'ALONE']
        #a_class = ['A', 'ABANDON', 'ABORTION', 'ABOVE', 'ACCEPT']
        #a_class = ['CHILD', 'LITTLE-BIT', 'LONELY', 'OLD', 'TWENTY'] 
        #a_class = ['OTHER', 'TWENTY', 'OLD', 'AND', 'LITTLE-BIT' ,'BACHELOR'] # robust - 10% of video
        # 5/7 - robust, 6/8 - okish k=1, 
        # 8/10 k=1, not robust, 10%
        #a_class = ['OTHER', 'TWENTY', 'OLD', 'AND', 'LITTLE-BIT' ,'BACHELOR', 'AIRPLANE', 'LONELY', 'ALONE', '#ALL']
        a_class = ['OTHER', 'TWENTY', 'OLD', 'AND', 'LITTLE-BIT' ,'BACHELOR', 'AIRPLANE', 'LONELY', 'ALONE', '#ALL', 'A']#, 'CHILD', 'THINK', 'PLEASE']
        #a_class = ['OTHER', 'TWENTY', 'AND', 'OLD' ,'BACHELOR', 'AIRPLANE', 'LONELY', 'ALONE', '#ALL', 'A', 'THINK', 'PLEASE']
        if not hasattr(self, 'loaded_data'):
            self.loaded_data_output =  []
            self.loaded_data = []

            for index, option1 in enumerate(a_class):
                data1, size_of_1 = self.load_vids_from_paths('/home/rakshak/Desktop/FYP/ASL_dataset/', option1)
                self.loaded_data_output = self.loaded_data_output + [index]*size_of_1 
                

                self.loaded_data = self.loaded_data + data1

        for_saving = dict()
        for_saving['Classes'] = a_class
        for_saving['loaded_data'] = self.loaded_data
        for_saving['loaded_data_output'] = self.loaded_data_output

        if True:
            with open('train_for_gsss.pkl', 'wb') as fp:
                pickle.dump(for_saving, fp)

        for i in range(len(a_class)):
            print('Class '+str(i)+'is '+a_class[i])

        print('------------------'+'Loaded Data One Hot'+'------------------')
        print(self.loaded_data_output)
        print('------------------'+'--------------------'+'------------------')

        if a_class_option:
            a_class = [a_class[a_class_option]]

        self.run(a_class, self.loaded_data, self.loaded_data_output)

    def customKNN(self, test_data, loaded_data, loaded_data_output):
        values = []
        for i in loaded_data:
            values.append(self.dis_sep(test_data, i, retain_list=False))
        ans = [x for _,x in sorted(zip(values, loaded_data_output))]
        return ans
        

    def run(self, a_class, loaded_data, loaded_data_output, real_time=False, real_time_data=None):
        ans_map = dict()

        if not real_time:
            for j in range(len(a_class)):
                expected = a_class[j]
                name_test = {'A':'Naomi',
                               'BACHELOR': 'Liz_2',
                                'AIRPLANE': 'Lana',
                                'OTHER': 'Naomi',
                                'ADULT-TALL': 'Liz_1',
                                'ALONE': 'Dana',
                                'AND': 'Liz',
                                '#ALL': 'Liz',
                                'ABANDON': 'Liz',
                                'ABORTION': 'Liz_1',
                                'ABOVE': 'Tyler_1',
                                'ACCEPT': 'Naomi',
                                'CHILD': 'Liz_1',
                                'LITTLE-BIT': 'Naomi',
                                'LONELY': 'Liz',
                                'OLD': 'Liz',
                                'TWENTY': 'Naomi',
                                'CRASH': 'Liz',
                                'ENTER': 'Liz_1',
                                'MILLION': 'Liz',
                                'THINK': 'Liz',
                                'MISTAKE': 'Liz',
                                'PLEASE': 'Liz',
                                'IDEA': 'Liz'}
                test_data = self.load_data(['/home/rakshak/Desktop/FYP/ASL_dataset/##testData/{}/{}.mov'.format(expected, name_test[expected])])
                ans_map[expected] = self.customKNN(test_data[0], loaded_data, loaded_data_output)
                
                #print(expected, sorted(values))
        else:
            ans_map['Test Data'] = self.customKNN(real_time_data, loaded_data, loaded_data_output)

        for i in range(1):
            print('=========================================================================================')
            k = i*2+1
            #print('K is ',k)
            #print()
            for expected, ans in ans_map.items():
                outcome = self.output_class(ans,k, len(a_class))
                
                

                #print("Arr", ans)
                print()
                if not real_time:
                    print("Expected Class: ", expected)
                if outcome == -1:
                    print('Output Class: No clear class')
                else:
                    print("Output Class: ", a_class[outcome])

                if real_time:
                    return a_class[outcome]

    def output_class(self, ans, k, length):
        arr = [0]*length
        for i in range(k):
            arr[ans[i]] += 1

        max_index = -1
        max_value = 0
        draw = False
        for i in range(length):
            if arr[i] == max_value:
                draw = True
            if arr[i] > max_value:
                max_value = arr[i]
                max_index = i
                draw = False

        if draw:
            return -1
        return max_index


    def get_list_of_vids_from_folder(self, path):

        cwd = os.getcwd()
        os.chdir(path)
        files = os.listdir()
        os.chdir(cwd)
        vids_path = []
        for i in files:
            if '.pkl' in i:# or 'Tyler' in i:# or 'Brady' in i:
                continue
            if '.mov' or '.avi' in i:
                #print(i)
                p = r'{}'.format(os.path.join(path, i))
                vids_path.append(p)
                if ONE_SHOT_LEARNING:
                    break
        return vids_path

    def save_as_pickle(self, path, data):
        with open(path, 'wb') as fp:
            pickle.dump(data, fp)
        fp.close()

    def load_from_pickle(self, path):
        with open(path, 'rb') as fp:
            data = pickle.load(fp)
        fp.close()
        return data

    def real_time(self, path=0):
        cap = cv2.VideoCapture(path)#'/home/rakshak/Desktop/FYP/proj/FYP src/Hand Tracking/continous_2.avi')#0)
        cnt = 0
        points_list=[]
        fps = 0
        train = self.load_from_pickle('train.pkl')

        print(train['Classes'])
        a_class = train['Classes']
        loaded_data = train['loaded_data'] 
        loaded_data_output = train['loaded_data_output']

        while(True):
            
            if cnt == 5:
                if len(points_list) > 5:
                    length = len(points_list)
                    percent = 0.3
                    amount = int(length*(percent/2))
                    print('amount', amount)
                    print('taken', length-2*amount)
                    cut_part = points_list[amount:length-amount]
                    self.run(a_class, loaded_data, loaded_data_output, real_time=True, real_time_data=np.array(cut_part))

                cnt = 0
                points_list=[]
                print('Reset')

            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame,(320,240))
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            t_frame = np.dstack([gray_frame,gray_frame,gray_frame])
            kp,box = self.hand_point_gen(t_frame)#frame[:,:,::-1]) # <=================== change here 
            normalized_kp = 0
            #print(box)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if kp is not None:
                if VERBOSE:
                    print(kp.shape)

                points_list.append(kp)

                if DISPLAY_LANDMARKS:
                    try:
                        cv2.rectangle(frame,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,255,0),3)
                        for i in range(len(kp[:,0])):
                            if VERBOSE:
                                print(kp[i,1],kp[i,0])
                            frame[int(kp[i,1]),int(kp[i,0])] = (0,0,255)
                            cv2.circle(frame,(int(kp[i,0]),int(kp[i,1])),5,(255,0,0),1)
                            cv2.circle(frame,(int(box[0][0]),int(box[0][1])),5,(0,0,255),1)
                            cv2.circle(frame,(int(((box[2][0]+box[0][0])//2)),int(((box[2][1]+box[0][1])//2))),5,(0,0,255),1)
                            #cv2.circle(frame,(int(box[2][1]),int(box[0][1])),5,(255,0,0),1)
                    except Exception as e:
                        print(e)
            else:
                cnt += 1

                
            cv2.imshow('frame', frame)

        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()
        #print(cnt)
        return points_list

    def custom_vids(self):
        train = self.load_from_pickle('/content/drive/My Drive/train_for_gsss.pkl')#'train_normal.pkl')

        for index,classes in enumerate(train['Classes']):
            print(classes, index)
        a_class = train['Classes']
        loaded_data = train['loaded_data'] 
        loaded_data_output = train['loaded_data_output']

        test_data, size_of, classes = self.load_vids_from_paths('/content/drive/My Drive/', 'custom_test', custom=True)
        recorder = dict()
        print(classes)
        for index, data in enumerate(test_data):
            expected_class = classes[index].split('_')[0]
            print("Expected Class:", classes[index])#expected_class)
            ans = self.run(a_class, loaded_data, loaded_data_output, real_time=True, real_time_data=data)
            print()

            if expected_class in recorder:
                recorder[expected_class].append(ans.lower())
            else:
                recorder[expected_class] = [ans.lower()]
        print()
        correct = 0
        total = 0
        class_wise_output = dict()
        for expected, result in recorder.items():
            print(expected, result)
            correct += result.count(expected)
            total += len(result)

            class_wise_output[expected] = dict()
            class_wise_output[expected]['Correct'] = result.count(expected)
            class_wise_output[expected]['False Positive'] = len(result) - class_wise_output[expected]['Correct']

        print(class_wise_output)

        print('gesture correct false-positive acc')
        for gesture in class_wise_output.keys():
            tot = class_wise_output[gesture]['Correct'] + class_wise_output[gesture]['False Positive']
            print(gesture, class_wise_output[gesture]['Correct'], class_wise_output[gesture]['False Positive'], ((class_wise_output[gesture]['Correct']/tot)*100) )


        print("Acc", ((correct/total)*100))

    def custom_capture(self):
        train = self.load_from_pickle('train_normal.pkl')

        print(train['Classes'])
        a_class = train['Classes']
        loaded_data = train['loaded_data'] 
        loaded_data_output = train['loaded_data_output']
        print(loaded_data_output)

        ok = 'n'
        name = input('Enter name of vid\n')

        while ok != 'y':
            take_vid(name)

            data = self.load_data(['/home/rakshak/Desktop/FYP/proj/FYP src/Hand Tracking/'+name+'.avi'])
            self.run(a_class, loaded_data, loaded_data_output, real_time=True, real_time_data=data[0])
            print()
            ok = input('Is it ok?')

    def ISL(self):

        class_list, output_list, output_mapper = self.GetISLData()
        train_dict = {
            'class_list': class_list,
            'output_list': output_list,
            'output_mapper': output_mapper
        }
        if True:
            with open('train_ISL.pkl', 'wb') as fp:
                pickle.dump(train_dict, fp)
        print(output_list)
        print(output_mapper)


    def GetISLData(self):
        from natsort import natsorted

        # vid_list(x) aka class_list => points_list
        # output_list

        class_list = []
        output_list = []
        output_mapper = dict()
        cnt = -1
        starting_path = '/content/drive/My Drive/One Handed/'


        for classes in os.listdir(starting_path):
            for topName in os.listdir(starting_path+classes):
                cnt += 1
                output_mapper[topName] = cnt 
                path = starting_path+classes+'/'+topName+'/Gallery_list/'
                print(topName)
                for s in os.listdir(path):
                    print('        '+s)
                    points_list = []
                    for frame in natsorted(os.listdir(path+s)):

                        #print(classes,s)
                        img = cv2.imread(path+s+'/'+frame)

                        rame = cv2.resize(img,(320,240))
                        # write the flipped frame
                        kp,box = self.hand_point_gen(img[:,:,::-1])
                        if kp is not None:
                            if DISPLAY_LANDMARKS:
                                try:
                                    cv2.rectangle(img,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,255,0),3)
                                    for i in range(21):
                                        img[int(kp[i,1]),int(kp[i,0])] = (0,0,255)
                                        cv2.circle(img,(int(kp[i,0]),int(kp[i,1])),5,(255,0,0),1)
                                        cv2.circle(img,(int(box[0][0]),int(box[0][1])),5,(0,0,255),1)
                                        cv2.circle(img,(int(((box[2][0]+box[0][0])//2)),int(((box[2][1]+box[0][1])//2))),5,(0,0,255),1)
                                        #cv2.circle(frame,(int(box[2][1]),int(box[0][1])),5,(255,0,0),1)
                                except Exception as e:
                                    print(e)

                            #cv2.imshow('frame',img)
                            if cv2.waitKey(1) & 0xFF == ord('q'):
                                break

                            points_list.append(kp)

                    class_list.append(points_list)
                    output_list.append(cnt)


                # Release everything if job is finished
                cv2.destroyAllWindows()

        return class_list, output_list, output_mapper







m = MainPart()
#m.customMain()
#m.real_time('/home/rakshak/Desktop/FYP/proj/FYP src/Hand Tracking/continous_2.avi')#0)
#m.custom_vids()
#m.custom_capture()
m.ISL()